### Model Registry

In [19]:
import boto3
import botocore

sm_client = boto3.client("sagemaker")

In [5]:
model_package_group_name='xgboost-abalone-training'
model_package_description = 'Model to predict the age of abalone from physical measurements'

In [9]:
image_uri = "366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.3-1"

In [10]:
modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": image_uri,
         }
      ],
      "SupportedContentTypes": [ "application/x-amz-json-1.1" ],
      "SupportedResponseMIMETypes": [ "application/x-amz-json-1.1" ],
   }
 }


In [25]:
#TODO: specify the model source
model_url = ""

In [26]:
# Specify the model data
create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : model_package_description,
    "ModelApprovalStatus" : "PendingManualApproval"
}

In [27]:
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

In [28]:
create_model_package_input_dict.update(modelpackage_inference_specification)

In [31]:
try:
    create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
except botocore.exceptions.ClientError as ce:
    # When model package group does not exit
    if ce.operation_name == "CreateModelPackage":
        if ce.response["Error"]["Message"] == "Model Package Group does not exist.":
            # Create model package group
            create_model_package_group_response = sm_client.create_model_package_group(
                ModelPackageGroupName=model_package_group_name,
                ModelPackageGroupDescription=model_package_description,
            )
            
            create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)

In [32]:
create_mode_package_response

{'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:687314952804:model-package/xgboost-abalone-training/1',
 'ResponseMetadata': {'RequestId': 'd6bf541c-d208-44fc-a6bc-029a3ba83781',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd6bf541c-d208-44fc-a6bc-029a3ba83781',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '108',
   'date': 'Tue, 18 Jan 2022 14:06:38 GMT'},
  'RetryAttempts': 0}}

In [1]:
#TODD: create a lambda function